# HowTo -Undistorted- Undistort RGB images using camera model

In this notebook you will learn:

1. how to connect to a ROMI database
2. how to use estimated camera intrinsic parameters to undistort images
3. visualize the output and compare it the original image

This notebook **assume** that you have:
- declared the `ROMI_DB` environment variable as the path to the database directory to use
- processed the test dataset with the _geometric pipeline,_ so we can access the fileset containing the data we want to start with...

Remember, the aim of this notebook is to show you how it works "under the hood".
This is not how you should process your data, that is done thanks to the `romi_run_task` CLI tool.

In [ ]:
import os

import ipywidgets as widgets
import numpy as np
from IPython.display import display
from plant3dvision.camera import get_camera_arrays_from_params, get_camera_kwargs_from_params_list
from plant3dvision.proc2d import undistort
from plant3dvision.visu import plotly_image_carousel
from plantdb import FSDB
from plantdb.io import read_image

## Connect to the database & get the initial data

If you did not declare a `ROMI_DB` environment variable, you can do it by uncommenting the next cell and setting it to the right value.

In [ ]:
# os.environ['ROMI_DB'] = "/path/to/test/data"

### Connect to the database

In [ ]:
db = FSDB(os.environ['ROMI_DB'])  # requires definition of this environment variable!
db.connect()

Once you are connected to the database, you can list the available scan *dataset* with `db.list_scans()`.

### Select a dataset

We now select a dataset (with the `Dropdown` widget) for the demo:

In [ ]:
scan_name = widgets.Dropdown(options=db.list_scans(), value=db.list_scans()[0], description='Dataset:')
display(scan_name)

In [ ]:
scan = db.get_scan(scan_name.value)

If you did not process this dataset yet, from the `plant3dvision` root directory, you can do it with:
```
romi_run_task AnglesAndInternodes $ROMI_DB/<selected_dataset> --config plant-3d-vision/configs/geom_pipe_real.toml
```

To list the available *filesets* in this *scan dataset*:

In [ ]:
scan.list_filesets()

### Get the RGB images fileset

The RGB images resulting from a scan by the _Plant Imager_ are to be found in the 'images' fileset.

In [ ]:
img_fs = scan.get_fileset("images")

Once you have access to the 'images' fileset, you may access the RGB images as follows:

In [ ]:
img_files = img_fs.get_files(query={"channel": "rgb"})

In [ ]:
print(f"This fileset contains {len(img_files)} files (matching the `query`).")

### Visualize the RGB images fileset

It is possible to visualize the set of RGB images using our `plotly_image_carousel` method.

In [ ]:
fig = plotly_image_carousel(img_files, title=scan_name.value)

In [ ]:
fig.show()

## Undistort an image

If you want more details about the theory and algorithms used to undistort images, have a look at the [OpenCV tutorial on camera calibration](https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html).

### Get estimated camera model

Estimated camera intrinsic parameter are saved as image metadata:

In [ ]:
camera_model = img_files[0].get_metadata('colmap_camera')['camera_model']
print(camera_model)

The OpenCV method requires a _camera matrix_ and a _distortion vector_, those can be easily obtained with:

In [ ]:
model = camera_model['model']
params = get_camera_kwargs_from_params_list(model, camera_model['params'])
camera_mtx, distortion_vect = get_camera_arrays_from_params(**params)

In [ ]:
camera_mtx

In [ ]:
distortion_vect

### Load the image

Using the `read_image` method from `plantdb.io`, you can load an image from a database `File`:

In [ ]:
img = read_image(img_files[0])

In [ ]:
type(img)

### Undistort the image with OPENCV

Using the `undistort` method from `plant3dvision.proc2d`, you can apply the estimated camera intrinsics as follows:

In [ ]:
undistorted_img = undistort(img, camera_mtx, distortion_vect)

In [ ]:
type(undistorted_img)

### Compare images with a split-view

In [ ]:
heigth, width, _ = img.shape
print(heigth, width)

In [ ]:
from PIL.Image import fromarray

split_width = 2
s = split_width // 2
split_im = np.zeros_like(img)


def split_image(x):
    split_im[:, :x - s, :] = img[:, :x - s, :]
    split_im[:, x - s:x + s, :] = [255., 0., 0.]
    split_im[:, x + s:, :] = undistorted_img[:, x + s:, :]
    #    return plt.imshow(split_im)
    return display(fromarray(split_im))


_ = widgets.interact(split_image, x=widgets.IntSlider(width // 2, min=s, max=width - s - 1))

As you may notice, the lens deformation is not dramatic!
Indeed, the Sony RX-0 is not a cheap camera and have a good Zeiss lens showing little to no deformation.

Things might be a bit different when using cheaper optics like the Raspberry Pi camera and the numerous clones out there.

We may now **disconnect** from the database as we will not need it anymore:

In [ ]:
db.disconnect()